In [1]:
import numpy as np

import torch
from torch import nn
from torch import optim
from torch.nn import functional as F

In [2]:
torch.manual_seed(1)

In [3]:
def split(examples):
    return [(s.split(), l) for s, l in examples]

In [4]:
data = split([
    ('me gusta comer en la cafeteria', 'SPANISH'),
    ('Give it to me', 'ENGLISH'),
    ('No creo que sea una buena idea', 'SPANISH'),
    ('No it is not a good idea to get lost at sea', 'ENGLISH')
])

In [5]:
test_data = split([
    ('Yo creo que si', 'SPANISH'),
    ('it is lost on me', 'ENGLISH')
])

In [6]:
word_to_index = {}
for sentence, _ in data + test_data:
    for word in sentence:
        if word not in word_to_index:
            word_to_index[word] = len(word_to_index)
print(word_to_index)

{'me': 0, 'gusta': 1, 'comer': 2, 'en': 3, 'la': 4, 'cafeteria': 5, 'Give': 6, 'it': 7, 'to': 8, 'No': 9, 'creo': 10, 'que': 11, 'sea': 12, 'una': 13, 'buena': 14, 'idea': 15, 'is': 16, 'not': 17, 'a': 18, 'good': 19, 'get': 20, 'lost': 21, 'at': 22, 'Yo': 23, 'si': 24, 'on': 25}


In [7]:
label_to_index = {'SPANISH': 0, 'ENGLISH': 1}

In [8]:
VOCAB_SIZE = len(word_to_index)
NUM_LABELS = 2

In [9]:
class BoWClassifier(nn.Module):
    
    def __init__(self, num_labels, vocab_size):
        super().__init__()
        self.linear = nn.Linear(vocab_size, num_labels)
        
    def forward(self, bow_vec):
        return F.log_softmax(self.linear(bow_vec), dim=1)

In [10]:
def make_bow_vector(sentence, word_to_index):
    vec = torch.zeros(len(word_to_index))
    for word in sentence:
        vec[word_to_index[word]] += 1
    return vec.view(1, -1)

In [11]:
def make_target(label, label_to_index):
    return torch.LongTensor([label_to_index[label]])

In [12]:
v = make_bow_vector('not a good sea'.split(), word_to_index)
v.shape

torch.Size([1, 26])

In [13]:
model = BoWClassifier(NUM_LABELS, VOCAB_SIZE)

In [14]:
def test(model, dataset):
    with torch.no_grad():
        for instance, label in dataset:
            bow_vec = make_bow_vector(instance, word_to_index)
            log_probs = model(bow_vec)
            print(instance, log_probs)

In [15]:
test(model, test_data)

['Yo', 'creo', 'que', 'si'] tensor([[-0.6250, -0.7662]])
['it', 'is', 'lost', 'on', 'me'] tensor([[-0.5870, -0.8119]])


In [16]:
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in range(100):
    running_loss = []
    
    for instance, label in data:
        model.zero_grad()
        bow_vec = make_bow_vector(instance, word_to_index)
        target = make_target(label, label_to_index)
        log_probs = model(bow_vec)
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()
        running_loss.append(loss.item())
    
    if epoch % 20 == 19:
        epoch_loss = np.mean(running_loss)
        print('Epoch: %03d - loss: %2.4f' % (epoch + 1, epoch_loss))

Epoch: 020 - loss: 0.0490
Epoch: 040 - loss: 0.0238
Epoch: 060 - loss: 0.0157
Epoch: 080 - loss: 0.0117
Epoch: 100 - loss: 0.0093


In [17]:
test(model, test_data)

['Yo', 'creo', 'que', 'si'] tensor([[-0.1210, -2.1721]])
['it', 'is', 'lost', 'on', 'me'] tensor([[-2.7767, -0.0643]])
